# Object-Level CND Annotations

This notebook demonstrates object-level CND annotations, allowing annotations to be applied to specific object instances rather than just their classes. This solves the core issue where users want to annotate individual objects (like sets) without modifying the class definition.

## 🎯 New Ergonomic API

You can now use decorators directly on objects in a Pythonic way!

In [1]:
import sys
from pathlib import Path

# Add the parent directory to the Python path
sys.path.append(str(Path().resolve().parent))

from spytial import (
    diagram, 
    orientation, cyclic, group, atomColor  # Now work on both classes AND objects!
)

## Demo 1: Set Grouping with Ergonomic API (Issue Example)

This demonstrates the core issue request: annotating specific objects (like sets) without modifying their classes.

### The Pythonic Way - Apply Decorators Directly to Objects!

In [2]:
# Create different sets - each can be annotated differently
fruits = {"apple", "banana", "cherry", "date"}
numbers = {1, 2, 3, 4, 5, 6, 7, 8, 9, 10}
colors = {"red", "green", "blue"}

# NEW ERGONOMIC WAY: Apply decorators directly to objects!
# This is much more Pythonic than separate annotate_* functions
fruits = atomColor(selector='items', value='orange')(
    group(field='items', groupOn=0, addToGroup=1)(fruits)
)

numbers = atomColor(selector='digits', value='blue')(
    group(field='digits', groupOn=0, addToGroup=1)(numbers)
)

# Leave colors set without special annotations

# Create a container to show them all
set_data = {
    "fruits": fruits,
    "numbers": numbers, 
    "colors": colors,
    "note": "fruits and numbers are grouped, colors is not"
}

print("✅ Successfully annotated different sets with object-specific annotations")
print("- fruits set: grouped with orange coloring")
print("- numbers set: grouped with blue coloring")
print("- colors set: default rendering")
print("\n🎯 Using the NEW ergonomic API - decorators work directly on objects!")

✅ Successfully annotated different sets with object-specific annotations
- fruits set: grouped with orange coloring
- numbers set: grouped with blue coloring
- colors set: default rendering

🎯 Using the NEW ergonomic API - decorators work directly on objects!


In [3]:
# Visualize the sets with their object-specific annotations
diagram(set_data)

## API Comparison: Old vs New

### Old Approach (Still Supported)
```python
from spytial import annotate_group, annotate_atomColor
annotate_group(my_set, field='items', groupOn=0, addToGroup=1)
annotate_atomColor(my_set, selector='items', value='orange')
```

### New Ergonomic Approach (Recommended)
```python
from spytial import group, atomColor
my_set = atomColor(selector='items', value='orange')(
    group(field='items', groupOn=0, addToGroup=1)(my_set)
)
```

**The new approach is more Pythonic** - decorators work the same way whether you're decorating a class or an object!

## Demo 2: Class + Object Annotations Working Together

This shows how class-level decorators and object-level decorators can combine seamlessly using the same decorator syntax.

In [4]:
# Class-level decorators (traditional usage)
@orientation(selector='left', directions=['left'])
@cyclic(selector='children', direction='clockwise')
class TreeNode:
    def __init__(self, value, left=None, right=None):
        self.value = value
        self.left = left
        self.right = right

# Create a tree structure
root = TreeNode("root")
root.left = TreeNode("left-child")
root.right = TreeNode("right-child")
root.left.left = TreeNode("left-left")
root.left.right = TreeNode("left-right")

# Object-level decorators (NEW ergonomic usage - same syntax!)
root = atomColor(selector='self', value='red')(root)

root.left = orientation(selector='self', directions=['below'])(
    group(field='subtree', groupOn=0, addToGroup=1)(root.left)
)

print("✅ Class annotations apply to ALL TreeNode instances")
print("✅ Object annotations apply only to specific instances")
print("✅ Both use the SAME decorator syntax!")
print("\n🎯 Much more intuitive - Python developers expect decorators to 'just work'")

✅ Class annotations apply to ALL TreeNode instances
✅ Object annotations apply only to specific instances
✅ Both use the SAME decorator syntax!

🎯 Much more intuitive - Python developers expect decorators to 'just work'


In [5]:
# Visualize the tree with mixed annotations
diagram(root)

## Demo 3: Built-in Types Annotations

Demonstrates annotations on various built-in types that can't normally store attributes.

In [6]:
# Built-in immutable types work perfectly with the ergonomic API

# Tuple (immutable)
coordinates = orientation(selector='axes', directions=['horizontal'])((10, 20, 30))

# Frozenset (immutable)
immutable_set = group(field='elements', groupOn=0, addToGroup=1)(frozenset([1, 2, 3, 4]))

# String (immutable) 
text = atomColor(selector='characters', value='purple')("Hello World")

# Combine in data structure
builtin_data = {
    "coordinates": coordinates,
    "immutable_set": immutable_set,
    "text": text,
    "info": "All these immutable types have object-specific annotations"
}

print("✅ Successfully annotated immutable built-in types:")
print(f"- tuple: {coordinates}")
print(f"- frozenset: {immutable_set}")
print(f"- string: '{text}'")
print("\n🎯 Even immutable types work with the ergonomic decorator syntax!")

✅ Successfully annotated immutable built-in types:
- tuple: (10, 20, 30)
- frozenset: frozenset({1, 2, 3, 4})
- string: 'Hello World'

🎯 Even immutable types work with the ergonomic decorator syntax!


In [7]:
# Visualize the built-in types with annotations
diagram(builtin_data)

## Demo 4: Object-Level Custom Providers

You can also set custom data providers for specific objects, controlling how they are serialized to atoms and relations.

In [8]:
from spytial import DataInstanceProvider, object_provider, set_object_provider

# Define a custom provider for sets
class CustomSetProvider(DataInstanceProvider):
    def can_handle(self, obj):
        return isinstance(obj, set)
    
    def provide_atoms_and_relations(self, obj, walker_func):
        obj_id = walker_func._get_id(obj)
        atom = {
            "id": obj_id,
            "type": "custom_set",
            "label": f"MySet[{len(obj)}]"
        }
        
        relations = []
        for i, item in enumerate(obj):
            item_id = walker_func(item)
            relations.append((f"element_{i}", obj_id, item_id))
        
        return atom, relations

# Method 1: Decorator syntax
my_set = {1, 2, 3, 4, 5}
my_set = object_provider(CustomSetProvider())(my_set)

# Method 2: Function call
my_other_set = {6, 7, 8}
set_object_provider(my_other_set, CustomSetProvider())

# Method 3: Combine with annotations
custom_annotated_set = object_provider(CustomSetProvider())(
    group(field='elements', groupOn=0, addToGroup=1)({10, 11, 12})
)

print("✅ Custom providers set for different sets")
print("- my_set: uses CustomSetProvider")
print("- my_other_set: also uses CustomSetProvider")
print("- custom_annotated_set: CustomSetProvider + group annotation")
print("\n🎯 Object-specific providers override default serialization!")

✅ Custom providers set for different sets
- my_set: uses CustomSetProvider
- my_other_set: also uses CustomSetProvider
- custom_annotated_set: CustomSetProvider + group annotation

🎯 Object-specific providers override default serialization!


In [9]:
# Create a comparison structure
provider_demo = {
    "normal_set": {"a", "b", "c"},  # Uses default provider
    "custom_set1": my_set,  # Uses custom provider
    "custom_set2": my_other_set,  # Uses custom provider
    "custom_annotated": custom_annotated_set  # Custom provider + annotations
}

# Visualize to see the difference
diagram(provider_demo)

### Custom Provider Benefits

- **Override default serialization**: Replace how specific objects are converted to atoms/relations
- **Object-specific behavior**: Different instances of the same class can use different providers
- **Combine with annotations**: Custom providers work alongside spatial annotations
- **Maintain type flexibility**: Keep the benefits of Python's dynamic typing while customizing visualization

In [10]:
# You can chain multiple decorators just like with classes
my_data = ["item1", "item2", "item3"]

# Chain multiple decorators in a readable way
annotated_data = atomColor(selector='items', value='green')(
    orientation(selector='items', directions=['vertical'])(
        group(field='elements', groupOn=0, addToGroup=1)(my_data)
    )
)

print("✅ Chained multiple decorators on a single object")
print(f"Result: {annotated_data}")

# Verify all annotations were applied
from spytial import collect_decorators
annotations = collect_decorators(annotated_data)
print(f"Applied {len(annotations['constraints'])} constraints and {len(annotations['directives'])} directives")

✅ Chained multiple decorators on a single object
Result: ['item1', 'item2', 'item3']
Applied 2 constraints and 1 directives


## Key Achievements

🎯 **CND annotations now work on OBJECTS, not just classes!**

- ✅ **Ergonomic API**: Use the same decorator syntax for both classes and objects
- ✅ **Pythonic**: Decorators work as Python developers expect them to
- ✅ **Unified**: No need to learn separate `annotate_*` functions
- ✅ **Flexible**: Works with built-in/immutable types (set, tuple, frozenset, str)
- ✅ **Combinable**: Class and object annotations combine seamlessly
- ✅ **Backward Compatible**: Existing class-level decorators work unchanged

### The Pythonic Way

```python
# Apply decorators to objects just like you would to classes
my_set = group(field='items', groupOn=0, addToGroup=1)(my_set)
my_obj = atomColor(selector='self', value='blue')(my_obj)

# Chain them naturally
result = atomColor(selector='items', value='red')(
    orientation(selector='items', directions=['horizontal'])(my_data)
)
```

**This is much more intuitive than having separate `annotate_*` functions!**